In [1]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

In [2]:
import itertools
import torch

from tsvar.simulate import GrangerBuscaSimulator
from tsvar.models import WoldModelVariational, WoldModelVariationalFixedBeta

---

## Generate small toy example dataset

Define model parameters

In [3]:
dim = 3

baseline = np.array([0.1, 0.1, 0.2])

# adjacency[i,j] = magnitude of influence from i to j
adjacency = np.array([
    [0.3, 0.8, 0.1],
    [0.2, 0.3, 0.1],
    [0.2, 0.1, 0.0]
])

beta = 1.0

end_time = 5e4

for i, j in itertools.product(range(dim), repeat=2):
    print(f"w({j:d} -> {i:d}) = {adjacency[j,i]:.2f}")

w(0 -> 0) = 0.30
w(1 -> 0) = 0.20
w(2 -> 0) = 0.20
w(0 -> 1) = 0.80
w(1 -> 1) = 0.30
w(2 -> 1) = 0.10
w(0 -> 2) = 0.10
w(1 -> 2) = 0.10
w(2 -> 2) = 0.00


Simulate a dataset

In [4]:
wold_sim = GrangerBuscaSimulator(mu_rates=baseline, Alpha_ba=adjacency, Beta_b=beta * np.ones((dim,)))
events = wold_sim.simulate(end_time, seed=42)
events = [torch.tensor(ev, dtype=torch.float) for ev in events]


print('Number of events per dimension:', list(map(len, events)))
print('Total number of events:', sum(map(len, events)))
print()
print(events[0])
print()
print(events[1])
print()
print(events[2])

Number of events per dimension: [18311, 27760, 14246]
Total number of events: 60317

tensor([1.4935e+01, 2.0515e+01, 2.1992e+01,  ..., 4.9994e+04, 4.9995e+04,
        4.9998e+04])

tensor([1.5358e+00, 4.4058e+00, 5.8989e+00,  ..., 4.9998e+04, 4.9998e+04,
        4.9999e+04])

tensor([4.5463e+00, 7.6132e+00, 8.3506e+00,  ..., 4.9997e+04, 4.9998e+04,
        4.9999e+04])


---

## Test variational inference algorithm

Create model object and set the data.

In [5]:
self = WoldModelVariational(verbose=True)
self.set_data(events)

Define the parameters of the prior.

In [6]:
as_pr = 1.0 * np.ones((dim + 1, dim))
ar_pr = 1.0 * np.ones((dim + 1, dim))

bs_pr = .0001 * np.ones((dim, dim))
br_pr = .0001 * np.ones((dim, dim))

zc_pr = [1.0 * np.ones((len(events[i]), dim+1)) for i in range(dim)]

In [7]:
self.fit(as_pr=as_pr, ar_pr=ar_pr, bs_pr=bs_pr, br_pr=br_pr, zc_pr=zc_pr, max_iter=100, tol=1e-5)

-------------------------------------------------- 0
Alpha posterior mean:
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
Z[0] posterior probabilities
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 ...
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
-------------------------------------------------- 1
Alpha posterior mean:
[[0.09 0.14 0.07]
 [0.25 0.39 0.19]
 [0.21 0.31 0.16]
 [0.27 0.4  0.2 ]]
Z posterior probabilities
[[0.04244 0.31839 0.34033 0.29885]
 [0.18102 0.50739 0.05035 0.26124]
 [0.50541 0.11299 0.30231 0.07929]
 ...
 [0.34969 0.23592 0.29148 0.12291]
 [0.53639 0.19715 0.17868 0.08778]
 [0.32047 0.27836 0.31214 0.08904]]
-------------------------------------------------- 2
Alpha posterior mean:
[[0.16 0.29 0.12]
 [0.19 0.26 0.15]
 [0.16 0.2  0.13]
 [0.19 0.23 0.15]]
Z posterior probabilities
[[0.08544 0.30212 0.33924 0.2732 ]
 [0.33204 0.43865 0.03261 0.1967 ]
 [0.70011 0.06187 0.19919 0.03884]
 ...
 [0.54497 0.16379 0.21737 0.0

Z posterior probabilities
[[0.9971  0.0001  0.00281 0.     ]
 [0.99988 0.00004 0.00009 0.     ]
 [0.99979 0.      0.00021 0.     ]
 ...
 [0.9997  0.00001 0.00029 0.     ]
 [0.99988 0.00001 0.00011 0.     ]
 [0.99965 0.00001 0.00034 0.     ]]
-------------------------------------------------- 29
Alpha posterior mean:
[[0.37 0.56 0.27]
 [0.   0.   0.01]
 [0.   0.   0.02]
 [0.   0.   0.  ]]
Z posterior probabilities
[[0.9979  0.      0.0021  0.     ]
 [0.99993 0.      0.00007 0.     ]
 [0.99984 0.      0.00016 0.     ]
 ...
 [0.99978 0.      0.00022 0.     ]
 [0.99991 0.      0.00009 0.     ]
 [0.99975 0.      0.00025 0.     ]]
-------------------------------------------------- 30
Alpha posterior mean:
[[0.37 0.56 0.27]
 [0.   0.   0.01]
 [0.   0.   0.02]
 [0.   0.   0.  ]]
Z posterior probabilities
[[0.99847 0.      0.00153 0.     ]
 [0.99994 0.      0.00006 0.     ]
 [0.99989 0.      0.00011 0.     ]
 ...
 [0.99984 0.      0.00016 0.     ]
 [0.99994 0.      0.00006 0.     ]
 [0.99981 0.

Z posterior probabilities
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]
-------------------------------------------------- 63
Alpha posterior mean:
[[0.37 0.56 0.28]
 [0.   0.   0.  ]
 [0.   0.   0.02]
 [0.   0.   0.  ]]
Z posterior probabilities
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]
-------------------------------------------------- 64
Alpha posterior mean:
[[0.37 0.56 0.28]
 [0.   0.   0.  ]
 [0.   0.   0.02]
 [0.   0.   0.  ]]
Z posterior probabilities
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]
-------------------------------------------------- 65
Alpha posterior mean:
[[0.37 0.56 0.28]
 [0.   0.   0.  ]
 [0.   0.   0.02]
 [0.   0.   0.  ]]
Z posterior probabilities
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]
-------------------------------------------------- 66
Alpha poste

Alpha posterior mean:
[[0.37 0.56 0.28]
 [0.   0.   0.  ]
 [0.   0.   0.02]
 [0.   0.   0.  ]]
Z posterior probabilities
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]
-------------------------------------------------- 99
Alpha posterior mean:
[[0.37 0.56 0.28]
 [0.   0.   0.  ]
 [0.   0.   0.02]
 [0.   0.   0.  ]]
Z posterior probabilities
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]
-------------------------------------------------- 100
Alpha posterior mean:
[[0.37 0.56 0.28]
 [0.   0.   0.  ]
 [0.   0.   0.02]
 [0.   0.   0.  ]]
Z posterior probabilities
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]]


In [8]:
alpha_hat_mean = np.round(self._as_po / self._ar_po, 2)
baseline_hat = alpha_hat_mean[0,:]
adjacency_hat = alpha_hat_mean[1:,:]
beta_hat = np.round(self._br_po / (self._bs_po - 1), 2) - 1

print('Baseline:')
print('---------')
print('Ground truth:')
print(baseline)
print('Estimated:')
print(baseline_hat)
print()

print('Adjacency:')
print('---------')
print('Ground truth:')
print(adjacency)
print('Estimated:')
print(adjacency_hat)
print()

print('Beta:')
print('-----')
print('Ground truth:')
print(beta * np.ones((dim, dim)))
print('Estimated:')
print(beta_hat)

Baseline:
---------
Ground truth:
[0.1 0.1 0.2]
Estimated:
[0.37 0.56 0.28]

Adjacency:
---------
Ground truth:
[[0.3 0.8 0.1]
 [0.2 0.3 0.1]
 [0.2 0.1 0. ]]
Estimated:
[[0.   0.   0.  ]
 [0.   0.   0.02]
 [0.   0.   0.  ]]

Beta:
-----
Ground truth:
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
Estimated:
[[-3.76 -3.83 -3.61]
 [-3.27 -3.2   2.51]
 [-3.96 -3.87 -3.79]]


---

# Test variational inference algorithm *with fixed $\beta$*

In [10]:
model_fixed_beta = WoldModelVariationalFixedBeta(verbose=True)
model_fixed_beta.set_data(events)

In [11]:
as_pr = 1.0 * np.ones((dim + 1, dim))
ar_pr = 1.0 * np.ones((dim + 1, dim))

zc_pr = [1.0 * np.ones((len(events[i]), dim+1)) for i in range(dim)]

model_fixed_beta.fit(as_pr=as_pr, ar_pr=ar_pr, zc_pr=zc_pr, max_iter=100, tol=1e-5)

In [12]:
alpha_hat_mean = np.round(model_fixed_beta._as_po / model_fixed_beta._ar_po, 2)
baseline_hat = alpha_hat_mean[0,:]
adjacency_hat = alpha_hat_mean[1:,:]

print('Baseline:')
print('---------')
print('Ground truth:')
print(baseline)
print('Estimated:')
print(baseline_hat)
print()

print('Adjacency:')
print('---------')
print('Ground truth:')
print(adjacency)
print('Estimated:')
print(adjacency_hat)
print()

Baseline:
---------
Ground truth:
[0.1 0.1 0.2]
Estimated:
[0.09 0.11 0.19]

Adjacency:
---------
Ground truth:
[[0.3 0.8 0.1]
 [0.2 0.3 0.1]
 [0.2 0.1 0. ]]
Estimated:
[[0.31 0.79 0.1 ]
 [0.21 0.28 0.1 ]
 [0.2  0.12 0.02]]

